In [66]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector
from langchain.prompts.few_shot import (
    FewShotPromptTemplate,
    FewShotChatMessagePromptTemplate,
)
from langchain.prompts.example_selector import LengthBasedExampleSelector

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo-1106",
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

In [2]:
# dynamic하게 example들을 선택하는 방법 -> LengthBasedExampleSelector 사용
# 때로는 정말 많은 예제들이 존재할 수 있음 -> 어느정도의 example들을 골라서 prompt에 허용할 건지 정해야 함
# 많은 prompt는 더 많은 비용을 지불해야 하기 때문 -> 지불할 돈이 있더라도 때로는 모델에 알맞는 양이 있기 때문

# LengthBasedExampleSelector는 기본적으로 예제들을 형식화할 수 있고, 예제의 양이 얼마나 되는지 확인할 수 있음
# 그렇게 되면 설정해 놓은 세팅값에 따라 prompt에 알맞은 예제를 골라줄거임

# 커스텀 example selector
# 원하는대로 example들을 얼마나 허용할지 정할 수 있음 -> 유저의 로그인 여부 또는 유저가 사용하는 언어에 따라 얼마나 example들을 허용할지 정할 수 있음

In [3]:
examples = [
    {
        "question": "What do you know about France?",  # 28
        "answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
    },
    {
        "question": "What do you know about Italy?",  # 25
        "answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
    },
    {
        "question": "What do you know about Greece?",  # 27? 26?
        "answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
    },
]

In [48]:
example_template = """
Human: {question}
AI: {answer}
"""

example_prompt = PromptTemplate.from_template(example_template)

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,  # 형식화와 양이 얼마나 되는지 알아보기 위함
    max_length=80,  # 최대 예제 개수 -> 예제의 단어 수
)

prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,  # 예제들을 형식화하고
    suffix="Human : What do you know about {country}?",  # 형식화된 모든 예제 마지막에 나오는 내용 -> 한번 다 형식화가 끝나고 사용자의 질문은 무엇인지 나오게끔
    input_variables=["country"],  # 유효성 검사를 위해서 입력변수 지정
)

prompt.format(country="Germany")

'\nHuman: What do you know about France?\nAI: \nHere is what I know:\nCapital: Paris\nLanguage: French\nFood: Wine and Cheese\nCurrency: Euro\n\n\n\n\nHuman: What do you know about Italy?\nAI: \nI know this:\nCapital: Rome\nLanguage: Italian\nFood: Pizza and Pasta\nCurrency: Euro\n\n\n\n\nHuman: What do you know about Greece?\nAI: \nI know this:\nCapital: Athens\nLanguage: Greek\nFood: Souvlaki and Feta Cheese\nCurrency: Euro\n\n\n\nHuman : What do you know about Germany?'

In [65]:
num = 1
ex1_q = LengthBasedExampleSelector(
    examples=examples, example_prompt=example_prompt
).get_text_length(examples[num]["question"])
ex1_a = LengthBasedExampleSelector(
    examples=examples, example_prompt=example_prompt
).get_text_length(examples[num]["answer"])
print(ex1_q, ex1_a, ex1_q + ex1_a)

6 15 21


In [67]:
prompt.format(country="Brazil")

'\nHuman: What do you know about France?\nAI: \nHere is what I know:\nCapital: Paris\nLanguage: French\nFood: Wine and Cheese\nCurrency: Euro\n\n\n\n\nHuman: What do you know about Italy?\nAI: \nI know this:\nCapital: Rome\nLanguage: Italian\nFood: Pizza and Pasta\nCurrency: Euro\n\n\n\n\nHuman: What do you know about Greece?\nAI: \nI know this:\nCapital: Athens\nLanguage: Greek\nFood: Souvlaki and Feta Cheese\nCurrency: Euro\n\n\n\nHuman : What do you know about Brazil?'

In [71]:
from typing import Any, Dict


class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):  # 필수 메서드
        self.examples.append(example)

    def select_examples(self, input_variables):
        # 예제 중에 하나를 랜덤하게 return 하려고 함
        from random import choice

        return [choice(self.examples)]

In [75]:
example_template = """
Human: {question}
AI: {answer}
"""

example_prompt = PromptTemplate.from_template(example_template)

example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,  # 예제들을 형식화하고
    suffix="Human : What do you know about {country}?",  # 형식화된 모든 예제 마지막에 나오는 내용 -> 한번 다 형식화가 끝나고 사용자의 질문은 무엇인지 나오게끔
    input_variables=["country"],  # 유효성 검사를 위해서 입력변수 지정
)

prompt.format(country="Germany")

'\nHuman: What do you know about Italy?\nAI: \nI know this:\nCapital: Rome\nLanguage: Italian\nFood: Pizza and Pasta\nCurrency: Euro\n\n\n\nHuman : What do you know about Germany?'